In [1]:
import sys
sys.path.append("Scripts")

import pandas as pd
import os

from openpyxl import Workbook
from openpyxl.reader.excel import load_workbook

from datetime import datetime
timer_start_time = datetime.now()

In [2]:
from colors import Color, pattern_fill
from snapshot_structures import *
from header import format_header

from caseload_details import highlight_no_values, highlight_cm_switch, highlight_SEL, highlight_tsg_no_attendance
from checkins import highlight_no_checkins, highlight_exits
from sc_entries import highlight_sc_entries
from tier1 import highlight_zero_parents, highlight_tier1_notes, tier1_missing_activity

from tsg_minimum import highlight_initiative, highlight_activity

from tier2 import *
from ssp_goals import *
from attributes import *
from contacts import *

Split All-school file into individual school files

In [3]:
#--Need to add check for column names/order

#Save each tab from All School Snapshot as individual dataframe
cld_df = pd.read_excel('Snapshots\\All School Snapshot Prep Flow Output.xlsx', sheet_name='Caseload Details')
cld_df = cld_df.sort_values(by=['School', 'Case Manager', 'Student'])

check_in_df = pd.read_excel('Snapshots\\All School Snapshot Prep Flow Output.xlsx', sheet_name='Check-ins')
check_in_df = check_in_df.sort_values(by=['School', 'Case Manager', 'Student'])

sc_entries_df = pd.read_excel('Snapshots\\All School Snapshot Prep Flow Output.xlsx', sheet_name='Site Coordination Entries')
sc_entries_df['Date'] = pd.Categorical(sc_entries_df['Date'], month_order)
sc_entries_df = sc_entries_df.sort_values(by=['School', 'Date'])

tier1_df = pd.read_excel('Snapshots\\All School Snapshot Prep Flow Output.xlsx', sheet_name='Tier 1')
tier1_df = tier1_df.sort_values(by=['School', 'Entered By', 'Start Date'])

# The following dataframes are based on sheets that may only be seasonally available
tier2_df = None
goal_check_df = None
attr_check_df = None
contact_df = None

try:
    tier2_df = pd.read_excel('Snapshots\\All School Snapshot Prep Flow Output.xlsx', sheet_name='Tier II')
    tier2_df = tier2_df.sort_values(by=['School', 'Entered By', 'Support Date'])
except:
    pass
try:
    goal_check_df = pd.read_excel('Snapshots\\All School Snapshot Prep Flow Output.xlsx', sheet_name='SSP Goals')
    goal_check_df = goal_check_df.sort_values(by=['School', 'Case Manager', 'Student'])
except:
    pass
try:
    attr_check_df = pd.read_excel('Snapshots\\All School Snapshot Prep Flow Output.xlsx', sheet_name='Attributes Check')
    attr_check_df = attr_check_df.sort_values(by=['School', 'Case Manager', 'Student'])
except:
    pass
try:
    contact_df = pd.read_excel('Snapshots\\All School Snapshot Prep Flow Output.xlsx', sheet_name='Contact Info')
    contact_df = contact_df.sort_values(by=['School', 'Case Manager', 'Student'])
except:
    pass



In [4]:
# Save all-school with sorted dataframes
with pd.ExcelWriter(f'Snapshots\\All School Snapshot Prep Flow Output.xlsx', date_format='M/D/YYYY', datetime_format='M/D/YYYY') as writer:
        cld_df.to_excel(writer, sheet_name= 'Caseload Details', index=False)
        check_in_df.to_excel(writer, sheet_name= 'Check-ins', index=False)
        sc_entries_df.to_excel(writer, sheet_name= 'Site Coordination Entries', index=False)
        tier1_df.to_excel(writer, sheet_name= 'Tier 1', index=False)     
        try:    
            tier2_df.to_excel(writer, sheet_name= 'Tier II', index=False)
        except:
            pass          
        try:    
            goal_check_df.to_excel(writer, sheet_name= 'SSP Goals', index=False)
        except:
            pass
        try:
            attr_check_df.to_excel(writer, sheet_name= 'Attributes Check', index=False)
        except:
            pass     
        try:    
            contact_df.to_excel(writer, sheet_name= 'Contact Info', index=False)
        except:
            pass


#Split all-school file into individual school files
schools = list(sc_entries_df.School.unique())
tier2_missing = []
fail_count = {'attributes': 0, 'goals': 0, 'tier2': 0, 'contacts': 0}
for school in schools:
    df1 = cld_df[cld_df.School == school]
    df2 = check_in_df[check_in_df.School == school]
    df3 = sc_entries_df[sc_entries_df.School == school]
    df4 = tier1_df[tier1_df.School == school]
    
    try:
        df5 = tier2_df[tier2_df.School == school]
    except:
        fail_count['tier2'] += 1
    try:
        df6 = goal_check_df[goal_check_df.School == school]
    except:
        fail_count['goals'] += 1
    try:
        df7 = attr_check_df[attr_check_df.School == school]
    except:
        fail_count['attributes'] += 1
    try:
        df8 = contact_df[contact_df.School == school]
    except:
        fail_count['contacts'] += 1

    
    with pd.ExcelWriter(f'Snapshots/Archive/school files/{school} Snapshot.xlsx', date_format='M/D/YYYY', datetime_format='M/D/YYYY') as writer:
        df1.to_excel(writer, sheet_name= 'Caseload Details', index=False)
        df2.to_excel(writer, sheet_name= 'Check-ins', index=False)
        df3.to_excel(writer, sheet_name= 'Site Coordination Entries', index=False)
        df4.to_excel(writer, sheet_name= 'Tier 1', index=False)     
        try:    
            df5.to_excel(writer, sheet_name= 'Tier II', index=False)
        except:
            pass          
        try:    
            df6.to_excel(writer, sheet_name= 'SSP Goals', index=False)
        except:
            pass
        try:
            df7.to_excel(writer, sheet_name= 'Attributes Check', index=False)
        except:
            pass     
        try:    
            df8.to_excel(writer, sheet_name= 'Contact Info', index=False)
        except:
            pass

for i in fail_count.keys():
    if fail_count[i] > 0:
        print(f"fail count for {i} is {fail_count[i]}")

fail count for attributes is 3
fail count for goals is 3
fail count for tier2 is 3
fail count for contacts is 3


Create function that applies each highlight function

In [5]:
#Adds highlights to cells
def monthly_format(book): 
    #turn the dataframes into sheets
    cld = book['Caseload Details']
    cie = book['Check-ins']
    sce = book['Site Coordination Entries']
    tr1 = book['Tier 1']
    
    highlight_no_values(cld)
    highlight_SEL(cld)
    highlight_tsg_no_attendance(cld)
    highlight_cm_switch(cld)
    
    highlight_no_checkins(cie)
    highlight_exits(cie)
    
    highlight_sc_entries(sce)
    
    highlight_zero_parents(tr1)
    highlight_tier1_notes(tr1)
    tier1_missing_activity(tr1)
    

In [6]:
# Create function that applies quarterly functions
#--Need to update functions to take in sheet instead of book--
quarterly_tally = {'SSP': 0, 'Attributes': 0, 'Tier II': 0, 'Contacts': 0}

def quarterly_format(book):
    try:
        tier2_missing_activity(book)
        tier2_missing_notes(book)
    except:
        quarterly_tally['Tier II'] += 1
        
    try:
        ssp_less_than_2_goals(book)
        ssp_zero_baseline(book)
        ssp_no_SEL_goals(book)
        ssp_highlight_banned_goals(book)
    except:
        quarterly_tally['SSP'] += 1
    
    try:
        attributes_check(book)
    except:
        quarterly_tally['Attributes'] += 1
        
    try:
        highlight_missing_contact(book)
    except:
        quarterly_tally['Contacts'] += 1



Create folder using current date

In [7]:
# Create folder using today's date
path = 'Snapshots\\School Snapshots ' + today_str
try:
    os.mkdir(path)
except:
    pass


Format all-school snapshot and put it in dated folder

In [8]:
wb = load_workbook('Snapshots\\All School Snapshot Prep Flow Output.xlsx')
format_header(wb)
monthly_format(wb)
highlight_initiative(wb, all_school=True)
highlight_activity(wb, all_school=True)
quarterly_format(wb)

wb.save(path + '\\All School Snapshot '+ today_str + '.xlsx')

Format each individual school file and put them in dated folder

Remove various columns

In [9]:
for file in os.listdir('Snapshots\\Archive\\school files'):
    book = load_workbook('Snapshots\\Archive\\school files\\' + file)
    monthly_format(book)
    try:
        quarterly_format(book)
    except:
        pass
    highlight_initiative(book, school=file[:-14])
    highlight_activity(book, school=file[:-14])
    

    # Remove TSG columnss not in school's overview matrix
    # columns have to be removed in reverse order to maintain consistent indices
    for school in school_tsg:
        if school in file:
            removal_list = [x for x in initiatives if x not in school_tsg[school]]
            removal_list.reverse()
            for col in removal_list:
                book['Caseload Details'].delete_cols(cld_col_num[col], 1)    
    
    # Adjust column widths & save to file
    format_header(book)
    
    book.save(path + '\\' + file[:-5] + ' ' + today_str + '.xlsx')

quarterly_tally_sum = 0
for i in quarterly_tally:
    quarterly_tally_sum += quarterly_tally[i]
if quarterly_tally_sum:
    print('-'*16, 'Sheets with no data?', '-'*16)
    for sheet_name in quarterly_tally:
        if quarterly_tally[sheet_name] > 0:
            print(f"{sheet_name} page has {quarterly_tally[sheet_name]} files with no data")

---------------- Sheets with no data? ----------------
SSP page has 4 files with no data
Attributes page has 4 files with no data
Tier II page has 4 files with no data
Contacts page has 4 files with no data


In [10]:
timer_end_time = datetime.now()
print('Duration: {}'.format(timer_end_time - timer_start_time))

Duration: 0:00:01.735852
